In [1]:
import torch
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from PIL import Image
import os
import pandas as pd
import numpy as np


In [2]:
path = './MURA-v1.1/'


In [3]:
valid_image_paths_csv = "valid_image_paths.csv"
df_valid_data_paths = pd.read_csv(os.path.join(path,valid_image_paths_csv),dtype=str,header=None)
df_valid_data_paths.columns = ['image_path']

In [4]:
df_valid_data_paths['label'] = df_valid_data_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
df_valid_data_paths['category']  = df_valid_data_paths['image_path'].apply(lambda x: x.split('/')[2])
#df_valid_data_paths['dir'] =  df_valid_data_paths['image_path'].apply(lambda x: x.split('/')[1])
#df_valid_data_paths['patientId']  = df_valid_data_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))

In [5]:
df_valid_data_paths["label"] = df_valid_data_paths["label"].replace({'positive': 1, 'negative': 0})


In [6]:
from skimage.filters import butterworth
from skimage.filters import gaussian
from skimage import exposure



In [8]:
from torch.utils.data import Dataset
from skimage import img_as_ubyte
from skimage.color import rgb2gray
class MuraDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None, device='cpu'):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.device = device

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.item()
        
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        image = image.convert('RGB')
        
        # Convert PIL Image to numpy array
        image_np = np.array(image)
        
        image_gray = rgb2gray(image_np)
        
        # Apply adaptive histogram equalization
        image_gray = exposure.equalize_adapthist(image_gray, clip_limit=0.02)

        
        # Apply Butterworth filter
        image_butterworth = butterworth(image_gray)

        
        # Apply Gaussian filter
        image_gaussian = gaussian(image_gray)
        
        # Combine the two images into one (stack them along the last axis)
        # Create a 3 channel image from the butterworth and gaussian filtered images
        image_combined = np.stack([image_butterworth, image_gaussian, image_gray], axis=-1)
        
        image_combined = (image_combined - np.min(image_combined)) / (np.max(image_combined) - np.min(image_combined))
        
        # Convert to 8-bit image for PIL
        image_combined = img_as_ubyte(image_combined)
        
        # Convert numpy array back to PIL Image
        image = Image.fromarray(image_combined)
        
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        image = image.to(self.device)
        
        label = torch.tensor(label).to(self.device)

        return image, label


In [9]:
from torch.utils.data import DataLoader
from torchvision.transforms import v2

batchsize = 16
mean_values = [0.2665, 0.4648, 0.4648]# Calculated
std_values = [0.1079, 0.1650, 0.1666] # Calculated

val_transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
    v2.Normalize(mean=mean_values, std=std_values)

])

Mura_transform_valid = MuraDataset(df_valid_data_paths["image_path"], df_valid_data_paths["label"], transform=val_transform,device='cuda')
valid_loader = DataLoader(Mura_transform_valid, batch_size=batchsize)



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [29]:
#googleNet = torch.load('Models weights\googlenet_mura_Norma_p2LionOPT_40epoch.pth')
googleNet = torch.load('Models weights\googlenet_mura_Norma_p2LionOPT_40epoch.pth')
swin = torch.load('SWIN_PART3mura_LION_Normalization_Balanced20epoch.pth')
densenet = torch.load('Models weights\densenet_mura_phase_3_p2_20epoch.pth')

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch.nn.functional as F

# Switch the model to evaluation mode
googleNet.eval()
swin.eval()
#densenet.eval()

# Initialize lists to store all true labels and all predictions
all_labels = []
all_predictions = []
all_probabilities = []

# Disabling gradient calculation
with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(valid_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass for all three models
        outputs_googleNet = googleNet(images)
        outputs_swin = swin(images)
        #outputs_densenet = densenet(images)
        logits = outputs_swin.logits 
        # Apply softmax to get probabilities from logits
        probs_googleNet = F.softmax(outputs_googleNet, dim=1)
        #probs_densenet = F.softmax(outputs_densenet,dim=1)
        probs_swin = F.softmax(logits, dim=1)
        
        # Change it to numpy because of some objects issues
        swin_proba_np = probs_swin.cpu().numpy()
        googleNet_proba_np = probs_googleNet.cpu().numpy()
        #densenet_proba_np = probs_densenet.cpu().numpy()

        # Soft Voting: Average the probabilities
        averaged_probs = (googleNet_proba_np + swin_proba_np) / 2

        
        predicted = np.argmax(averaged_probs, axis=1)  # This will give you the indices of the maximum values along axis 1


        # Store predictions and true labels for later calculation of metrics
        all_predictions.extend(predicted.tolist())  # converted to a list before extending
        all_labels.extend(labels.cpu().numpy().tolist())  # converted to a list before extending
        all_probabilities.extend(averaged_probs.tolist())


        if (batch_idx + 1) % 100 == 0:
            print(f"Test Batch [{batch_idx+1}/{len(valid_loader)}] Processed")

# Compute metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
# change this to all_predictions or predicted
auc = roc_auc_score(all_labels, [proba[1] for proba in all_probabilities])


print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test AUC: {auc:.4f}")



Test Batch [100/200] Processed
Test Batch [200/200] Processed
Test Accuracy: 0.8180
Test Precision: 0.8205
Test Recall: 0.8180
Test F1-Score: 0.8171
Test AUC: 0.8716


# Test Accuracy: 0.8120
# Test Precision: 0.8215
# Test Recall: 0.8120
# Test F1-Score: 0.8097